<h1 style="background-color:#1c6ce6;
           font-family:segoe ui;
                color:#FFF9ED;
           font-size:300%;
           text-align:center;
           border-radius:10px 10px;"> 
🗺️ תהליך הסבת שכבת תשריטי תיעוד גבולות לבנק"ל מודרני 🗺️ 

In [ ]:
print(' Tatag borders:')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
ספריות 📚

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import geopandas as gpd
from configs import CNFG
from helpers import assign_BlockName, assign_ProcessName

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">קריאת נתונים 📑 

In [ ]:
print('   Reading inputs')

<h1 style="text-align:right;font-size:100%">
שכבת גבולות תת"ג קיימת

In [ ]:
tatag_cols = ['TATAG_NUM', 'TATAG_YEAR', 'TATAG_STATUS', 'SURVEYOR_NO', 'GUSH_NUM', 'GUSH_SUFFIX', 'geometry']

tatag_layer = gpd.read_file(CNFG.CADSDE + 'TATAG.gdb', layer = 'TATAG', include_fields = tatag_cols)

del tatag_cols

<h1 style="text-align:right;font-size:100%">
שמות תת"גים כפולים

In [ ]:
duplicated_names = pd.read_excel(CNFG.Local + 'tatag_duplicated_names.xlsx')['TatagName']

<h1 style="text-align:right;font-size:100%">
שכבת גבולות גושי קדסטר מודרנית

In [ ]:
Blocks = pd.read_csv(CNFG.CSVPath + 'BlockUniqueIDs.csv', usecols = ['Name', 'GlobalID', 'IsTax'])
Blocks = Blocks[Blocks['IsTax'] == 0].drop(columns = 'IsTax')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
עיבוד נתונים 🛠️

In [ ]:
print('   Cleaning data')

<h1 style="text-align:right;font-size:125%">
:עיבוד נתוני תת"ג
<h1 style="text-align:right;font-size:100%">
הסרת רשומות ללא מידע של גוש    
<h1 style="text-align:right;font-size:100%">
הוספת שדה מקשר של שם מלא לגוש לטובת איחוד עם שכבת גושים וקבלת מספר מזהה לגוש

In [ ]:
tatag_layer.dropna(subset = ['GUSH_NUM'], inplace=True)

tatag_layer = assign_BlockName(tatag_layer, 'GUSH_NUM', 'GUSH_SUFFIX')

tatag_layer = tatag_layer.merge(Blocks, left_on = 'BlockName', right_on='Name', how='left')\
                         .rename(columns = {'GlobalID' : 'BlockUniqueID'})

In [ ]:
print('   Data conversion')

<h1 style="text-align:right;font-size:100%">
הגדרת שדה של שם תכנית
<h1 style="text-align:right;font-size:100%">
מספרי תכנית שכבר קיימים בתור תכניוןתת מסוג אחר יקבלו קידומת 0 לשם על מנת לשמר ייחודיות

In [ ]:
tatag_layer = assign_ProcessName(tatag_layer, 'TATAG_NUM', 'TATAG_YEAR', drop=False)

tatag_layer['ProcessName'] = np.where(tatag_layer['ProcessName'].isin(duplicated_names),
                                      '0' + tatag_layer['ProcessName'],
                                      tatag_layer['ProcessName'])

del duplicated_names

<h1 style="text-align:right;font-size:100%">
מיפוי ערכי סטאטוס

In [ ]:
statusDict = {0 : 2,     #לא ידוע = בביקורת
              1 : 2,     #בביקורת = בביקורת
              2 : 13,    #מאושר = מאושר
              3 : 9,     #מחיקת תת"ג = מבוטל
              None : 2}  #לא ידוע = בביקרות

tatag_layer['Status'] = tatag_layer['TATAG_STATUS'].map(statusDict)

del statusDict

<h1 style="text-align:right;font-size:100%">
חישוב שדה רשת בקרה

In [ ]:
geodeticNetDict = {range(0,    1998)  : 1,     # רשת ישראל הישנה
                   range(1999, 2007)  : 2,     # רשת ישראל החדשה
                   range(2008, 99999) : 3}     # רשת ישראל התקפה

tatag_layer['GeodeticNetwork'] = tatag_layer['TATAG_YEAR'].apply(lambda x: next((v for k, v in geodeticNetDict.items() if x in k), 0))

del geodeticNetDict

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
יצירת שכבה חדשה בהתאם לאפיון ✨

In [ ]:
print('   Defining new Layer')

In [ ]:
new_columns = np.dtype([
                        ("geometry", gpd.array.GeometryDtype),       
                       #("GlobalID", str),
                        ("ProcessName", str),
                        ("ProcessType", int),
                        ("Status", int),
                        ("BlockUniqueID", str),
                        ("GeodeticNetwork", int),
                        ("SurveyorLicenseID", int),
                        ("DataSource", int),
                       #("PlanName", str),  
                       #("ORACLE_ID", int),
                       #("Shape_Length", float),
                       #("Shape_Area", float)
                        ])   

tatag_new = gpd.GeoDataFrame(np.empty(0, dtype=new_columns),
                             geometry = 'geometry',
                             crs = 2039)

<h1 style="text-align:right;font-size:100%">
איכלוס שדות

In [ ]:
#tatag_new['PlanName'] = None

tatag_new['ProcessName'] = tatag_layer['ProcessName']

tatag_new['geometry'] = tatag_layer['geometry']

tatag_new['ProcessType'] = 4 #תת"ג

tatag_new['DataSource'] = 5 # CAD

tatag_new['Status'] = tatag_layer['Status'].astype(int)

tatag_new['GeodeticNetwork'] = tatag_layer['GeodeticNetwork'].astype(int)

tatag_new['SurveyorLicenseID'] = tatag_layer['SURVEYOR_NO'].astype(int)

tatag_new['BlockUniqueID'] = tatag_layer['BlockUniqueID']

tatag_new = tatag_new.drop_duplicates(subset = 'ProcessName', keep='last')


#הסרת תכניות בעלי סטאטוס לא ידוע
tatag_new = tatag_new.dropna(subset=['Status'])

In [ ]:
tatag_new.sort_values('ProcessName', inplace = True)

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
שמירת תוצאות 💾
<h1 style="text-align:right; font-size:100%;">

<h1 style="text-align:right; font-size:100%;">     
פיצול התוצאות לפי סוגי גיאומטריה שונים (פעולה טכנית כי לא ניתן לכתוב קובץ סוגי גיאומטריה שונים באותה שכבה)    
<h1 style="text-align:right; font-size:100%;"> 
שמירת שכבה גיאוגרפית 

In [ ]:
print('   Exporting results')

In [ ]:
tatag_new.to_file(CNFG.Results, layer = 'PT4', driver = 'OpenFileGDB', engine = 'pyogrio', mode = 'w')